# 신경망 튜닝 방법

## 1. 학습률 감소 / 계획법 (Learning rate Decay / Scheduling)

### [1] 학습률 (Learning rate)

#### <1> 학습률이란?
+ 가중치의 기울기값을 경사하강법에 반영하는 정도
+ 학습의 보폭

#### <2> 학습률의 크기
+ 小 : 학습 진행이 느려, 학습 소요 시간이 길어짐
+ 大 : 최적값에 도달하지 못하고 주변을 맴돌거나, 심하면 오히려 발산 할 수 있음

### [2] 학습률 감소 (Learning rate Decay)

#### <1> 학습률 감소란
+ 학습이 진행되면서, 점차 학습률이 감소. (최적점에 가까워 질수록, 보폭을 줄여 못다가는 일이 업도록 하는 것.)
+ 주요 옵티마이저(Adagrad, RMSprop, Adam)에 이미 구현
+ model.compile() 에서 해당 옵티마이저 적용 후, 내부 하이퍼파라미터를 변경해 적용
```
model.compile(
    optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1 = 0.89),   
    loss = 'sparse_categorical_crossentropy',   
    metrics=['accuracy'])
```

### [3] 학습률 계획법 (Learning rate Scheduling)
+ 학습률을 계획적으로 지정하여 변경하는 방법
+ worm_up step : 학습률을 0에서부터 점차 증가시켜가는 구간
+ Decay step : cosine, step 등 다양한 방법으로 학습률 감소
```
first_decay_steps = 1000
initial_learning_rate = 0.01

lr_decayed_fn = (
  tf.keras.experimental.CosineDecayRestarts(
      initial_learning_rate,
      first_decay_steps))
      
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_decayed_fn)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
```

## 2. 가중치 초기화 (Weight Initialization)

### [1] 가중치 초기화란?
초기 가중치를 최적점에 가깝게 설정하는 방법

### [2] 가중치 초기화의 종류

#### <1> 표준편차가 1인 정규분포
+ 대부분의 활성화 값이 0 or 1 -> 학습이 정상적으로 이루어 지지 않음

#### <2> Xavier-Glorot 초기화
+ 이전층의 노드 수가 $n$ 일 때, 현재층의 가중치를 표준편차가 ${1 \over \sqrt{n}}$인 정규분포로 초기화
+ 표준편차가 고정된 경우 학습이 정상적으로 이루어지지 않는 점을 해결.(n은 이전과 현재 층의 노드를 같이 쓰는 등 변화 가능)
+ Sigmoid 활성화 함수에서는 잘 작동하나, ReLU 활성화 함수에서는 층이 지날수록, 활성화 값이 편중됨. -> 학습이 정상적으로 이루어 지지 않음

#### <3> He 초기화
+ 이전층의 노드 수가 $n$ 일 때, 현재층의 가중치를 표준편차가 ${2 \over \sqrt{n}}$인 정규분포로 초기화
+ ReLU를 사용할 경우 층이 깊어질수록 학습이 정상적으로 이루어지지 않는 점을 해결.

### [3] 초기화 방법
```
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
```

## 3. 과적합 (Overfitting)
train 데이터에 과하게 최적화되어, 일반적인 성능이 떨어지는 현상을 말하며, 층이 증가함에 따라 매개 변수가 기하급수적으로 증가하는 인공지능에서는 더 쉽게 발생함.

### [1] 가중치 감소 (Weight Decay)
+ 손실 함수에 가중치 관련항을 넣어, 가중치가 너무 커지는 것을 방지.
+ L1 Regularization(LASSO) : $L_1(\theta_w) = {1 \over 2}\sum_i(output_i - target_i)^2 + \lambda \cdot ||\theta_w||_1$
+ L2 Regularization(Ridge) : $L_2(\theta_w) = {1 \over 2}\sum_i(output_i - target_i)^2 + \lambda \cdot ||\theta_w||_2$
```
Dense(64,
      kernel_regularizer=regularizers.l2(0.01),
      activity_regularizer=regularizers.l1(0.01))
```

### [2] Dropout 
+ Iteration 마다 각 노드가 일정 확률로 사용되지 않도록 한다.
```
Dense(64,
      kernel_regularizer=regularizers.l2(0.01),
      activity_regularizer=regularizers.l1(0.01))
Dropout(0.5)
```

### [3] 조기 종료 (Early Stopping)
+ 학습이 계속되면서, Train 데이터에 대한 손실은 줄어드는데, Validation 데이터에 대한 손실이 증가하면(과적합이 발생하기 시작하면) 학습을 종료하는 방법

# Fashion MNIST 예제

In [ ]:
# 구글 드라이브 마운트(cjyjob1993@gmail.com)
from google.colab import drive
drive.mount('/content/drive')

# lib 디렉토리를을 환경 변수에 추가
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/myCode/lib')

In [ ]:
# 라이브러리 임포트
from tensorflow.keras.datasets import fashion_mnist
# 커스텀 라이브러리 임포트
from func_debug_log import debug
import numpy_fixSeed
import tf_fixSeed

: 

In [ ]:
# debug_log (0 : 미출력, 1 : 함수 실행 로그 출력, 2 : 함수 내부)
debug_flag = 1

In [ ]:
if debug_flag in range(0, 2) : debug('데이터 불러오기', __name__)
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

: 